# Single Cell with Fragmented or Fused Vacuole

This notebook contains the Finite Element Modeling (FEM) analysis code used to investigate stress and strain distribution 
in a 2D mesh representing a single cell with fragmented or fused vacuole, segmented into three subdomains: the cell wall, the interface between cells, and the symplast.

> **Note**  
> This notebook is designed to run within a *fracvac_env* environment.   

### Libraries and Dependencies  

All relevant **bvpy** (v.1.3.0) functions and classes required for this analysis are loaded by sourcing `lib_fun`.  

> **Important**  
> File paths for input and output functions (`io_function`) are relative to the directory from which this notebook is executed. In this case, the notebook should be opened from the `notebook` folder.

In [ ]:
import sys
import os

# Add the directory where function.py is located to the Python path
sys.path.append(os.path.expanduser("../function/"))

from lib_fun import *

### Generating the Mesh

Once the required libraries are loaded, the next step is to generate the mesh.

> **Note**  
> - The `generate_mesh()` function uses **GMSH** to create `.msh` files from `.geo` files, which define the physical surface attributes.  
> - The `CustomDomainGMSH()` function reads the `.msh` files and converts the data into a format compatible with **bvpy**.

In [ ]:
mesh_file = '../../data/in/single_cell_fvac.geo'
initial_scale = 0.05
generate_mesh(mesh_file, initial_scale)
mesh_path = '../../data/in/single_cell_fvac.msh'
fvac = CustomDomainGmsh(fname=mesh_path)


In [ ]:
mesh_file = '../../data/in/single_cell_hvac.geo'
initial_scale = 0.05
generate_mesh(mesh_file, initial_scale)
mesh_path = '../../data/in/single_cell_hvac.msh'
hvac = CustomDomainGmsh(fname = mesh_path)

### Visualizing the Mesh

To ensure the mesh has been correctly loaded and interpreted, you can visualize it using the following command:

In [ ]:
plot(fvac.cdata)
plot(hvac.cdata)

### Locate the mesh nodes related to the boundary conditions

In [ ]:
import numpy as np
# Boundaries condition            

xmax = hvac.mesh.coordinates()[:, 0].max()
xmin = hvac.mesh.coordinates()[:, 0].min()
ymax = hvac.mesh.coordinates()[:, 1].max()
ymin = hvac.mesh.coordinates()[:, 1].min()
ymed = np.median(hvac.mesh.coordinates()[:, 1])

left_border = Boundary(f'near(x, {xmin})') 
right_border = Boundary(f'near(x, {xmax})') 
bottom_border = Boundary(f'near(y, {ymin})')
top_border = Boundary(f'near(y, {ymax})')

fix = Boundary(f'near(y, {ymed} )')
fixy = (right_border | left_border) & fix 

all_borders = Boundary('all')
vacuol_borders = all_borders & ~ left_border & ~ right_border & ~ top_border & ~ bottom_border

### Setting Boundary Conditions and Material Properties

In the following cell, we define the boundary conditions, material properties, and the governing equations for the simulation. These parameters control how the mesh responds to external forces, displacements, and internal material properties.

#### Boundary Conditions

The boundary conditions are implemented using:
- `dirichlet`: Fixes specific displacement values at defined boundaries.
- `NormalNeumann`: apply pressure.  

#### Material Properties
- **Poisson's Ratio (`poiss`)**: Specifies the material's lateral expansion relative to its longitudinal compression.  
- **Young's Modulus (`young_values_by_labels`)**: Defines the stiffness of each subdomain, with values assigned based on the subdomain label:
    - **1** is the cell wall domain
    - **2** is the symplast domain
    - **3** is the interface between cells

#### Governing Equations
- The **StVenantKirchoffPotential** is used to define the strain-energy density function ie. the material's nonlinear elastic behavior. This is essential for simulating realistic large deformations.
$$
\mathcal{E}_{\text{SVK}} = \frac{\lambda}{2} \text{tr}(\boldsymbol{E})^2 + \mu\text{tr}(\boldsymbol{E}^2)
$$

where $\lambda, \mu$ are coefficients, $\boldsymbol{E}$ stands for the **Euler-Lagrange** strain tensor and $\boldsymbol{F}$ the deformation gradient.
- Plane stress conditions are assumed for this 2D analysis.  

#### Nonlinear Boundary Value Problem (BVP)
The `BVP` class is used to set up and solve the system, combining the domain, variational formulation, and boundary conditions into a single framework. The `nl_fragmented` or `nl_whole` object encapsulates this setup for each vacuol configuration.

In [ ]:
poiss = 0.3
TP = 0.075
frac_pressure = [NormalNeumann(val=-TP*0.336977, boundary=vacuol_borders),
                 dirichlet(0, boundary=right_border, subspace=0),
                 dirichlet(0, boundary=right_border, subspace=2),
                 dirichlet(0, boundary=left_border, subspace=0),
                 dirichlet(0, boundary=left_border, subspace=2), 
                 dirichlet(0, boundary=fixy, subspace = 1)]

whole_pressure = [NormalNeumann(val=-TP, boundary=vacuol_borders),
                 dirichlet(0, boundary=right_border, subspace=0),
                 dirichlet(0, boundary=right_border, subspace=2),
                 dirichlet(0, boundary=left_border, subspace=0),
                 dirichlet(0, boundary=left_border, subspace=2), 
                 dirichlet(0, boundary=fixy, subspace = 1)]

young_values_by_labels = {1:100, 2:1,3:1}
f_heterogeneous_young = HeterogeneousParameter(fvac.cdata, young_values_by_labels)
h_heterogeneous_young = HeterogeneousParameter(hvac.cdata, young_values_by_labels)
elastic_potential_f = StVenantKirchoffPotential(young=f_heterogeneous_young, poisson=poiss)
elastic_potential_h = StVenantKirchoffPotential(young=h_heterogeneous_young, poisson=poiss)

f_heterogeneous_Hyperelastic_response = HyperElasticForm(potential_energy=elastic_potential_f, source=[0., 0., 0.],
                                                       plane_stress=True)

h_heterogeneous_Hyperelastic_response = HyperElasticForm(potential_energy=elastic_potential_h, source=[0., 0., 0.],
                                                       plane_stress=True)

nl_fragmented = BVP(domain=fvac, vform=f_heterogeneous_Hyperelastic_response, bc=frac_pressure)
nl_whole = BVP(domain=hvac, vform=h_heterogeneous_Hyperelastic_response, bc=whole_pressure)

### Solving the Nonlinear System

In this next step, we solve the nonlinear system representing the deformation of the mesh. The goal is to iteratively compute the equilibrium configuration of the mesh.

#### Solver Configuration:
- **`tolerance`**: Determines the precision of the solution. Both absolute and relative tolerances are set to `1e-9` to ensure high accuracy in the total convergence process. 
- **`nl.solve()`**: This method solves the nonlinear boundary value problem using iterative algorithms.  
- **`linear_solver='mumps'`**: Specifies the use of the mumps linear solver for efficient and robust performance.  

In [ ]:
tolerance = 1e-9
nl_fragmented.solve(linear_solver='mumps', krylov_solver={'absolute_tolerance':1e-13}, absolute_tolerance=tolerance, relative_tolerance=tolerance)
solu_frac = nl_fragmented.solution
xdmf_save(path='../../data/out/fvac_force.xdmf', solution=solu_frac, vform=f_heterogeneous_Hyperelastic_response)

nl_whole.solve(linear_solver='mumps', krylov_solver={'absolute_tolerance':1e-13}, absolute_tolerance=tolerance, relative_tolerance=tolerance)
solu_whole = nl_whole.solution
xdmf_save(path='../../data/out/hvac_force.xdmf', solution=solu_whole, vform=h_heterogeneous_Hyperelastic_response)

In [ ]:
plot(solu_frac, size = 1)
plot(solu_whole, size = 1)

In [ ]:

frac_pressure = [NormalNeumann(val=-TP, boundary=vacuol_borders),
                 dirichlet(0, boundary=right_border, subspace=0),
                 dirichlet(0, boundary=right_border, subspace=2),
                 dirichlet(0, boundary=left_border, subspace=0),
                 dirichlet(0, boundary=left_border, subspace=2), 
                 dirichlet(0, boundary=fixy, subspace = 1)]

whole_pressure = [NormalNeumann(val=-TP, boundary=vacuol_borders),
                 dirichlet(0, boundary=right_border, subspace=0),
                 dirichlet(0, boundary=right_border, subspace=2),
                 dirichlet(0, boundary=left_border, subspace=0),
                 dirichlet(0, boundary=left_border, subspace=2), 
                 dirichlet(0, boundary=fixy, subspace = 1)]

young_values_by_labels = {1:100, 2:1,3:1}
f_heterogeneous_young = HeterogeneousParameter(fvac.cdata, young_values_by_labels)
h_heterogeneous_young = HeterogeneousParameter(hvac.cdata, young_values_by_labels)
elastic_potential_f = StVenantKirchoffPotential(young=f_heterogeneous_young, poisson=poiss)
elastic_potential_h = StVenantKirchoffPotential(young=h_heterogeneous_young, poisson=poiss)

f_heterogeneous_Hyperelastic_response = HyperElasticForm(potential_energy=elastic_potential_f, source=[0., 0., 0.],
                                                       plane_stress=True)

h_heterogeneous_Hyperelastic_response = HyperElasticForm(potential_energy=elastic_potential_h, source=[0., 0., 0.],
                                                       plane_stress=True)

nl_fragmented = BVP(domain=fvac, vform=f_heterogeneous_Hyperelastic_response, bc=frac_pressure)
nl_whole = BVP(domain=hvac, vform=h_heterogeneous_Hyperelastic_response, bc=whole_pressure)   

tolerance = 1e-9
nl_fragmented.solve(linear_solver='mumps', krylov_solver={'absolute_tolerance':1e-13}, absolute_tolerance=tolerance, relative_tolerance=tolerance)
solu_frac = nl_fragmented.solution
xdmf_save(path='../../data/out/fvac_pressure.xdmf', solution=solu_frac, vform=f_heterogeneous_Hyperelastic_response)

nl_whole.solve(linear_solver='mumps', krylov_solver={'absolute_tolerance':1e-13}, absolute_tolerance=tolerance, relative_tolerance=tolerance)
solu_whole = nl_whole.solution
xdmf_save(path='../../data/out/hvac_pressure.xdmf', solution=solu_whole, vform=h_heterogeneous_Hyperelastic_response)

### Testing a range of Turgor pressure

In [ ]:
tolerance = 1e-9
for i in range(28):
    TP = (i+3)*0.0025

    frac_pressure = [NormalNeumann(val=-TP, boundary=vacuol_borders),
                 dirichlet(0, boundary=right_border, subspace=0),
                 dirichlet(0, boundary=right_border, subspace=2),
                 dirichlet(0, boundary=left_border, subspace=0),
                 dirichlet(0, boundary=left_border, subspace=2), 
                 dirichlet(0, boundary=fixy, subspace = 1)]

    whole_pressure = [NormalNeumann(val=-TP, boundary=vacuol_borders),
                 dirichlet(0, boundary=right_border, subspace=0),
                 dirichlet(0, boundary=right_border, subspace=2),
                 dirichlet(0, boundary=left_border, subspace=0),
                 dirichlet(0, boundary=left_border, subspace=2), 
                 dirichlet(0, boundary=fixy, subspace = 1)]

    young_values_by_labels = {1:100, 2:1,3:1}
    f_heterogeneous_young = HeterogeneousParameter(fvac.cdata, young_values_by_labels)
    h_heterogeneous_young = HeterogeneousParameter(hvac.cdata, young_values_by_labels)
    elastic_potential_f = StVenantKirchoffPotential(young=f_heterogeneous_young, poisson=poiss)
    elastic_potential_h = StVenantKirchoffPotential(young=h_heterogeneous_young, poisson=poiss)

    f_heterogeneous_Hyperelastic_response = HyperElasticForm(potential_energy=elastic_potential_f, source=[0., 0., 0.],
                                                       plane_stress=True)

    h_heterogeneous_Hyperelastic_response = HyperElasticForm(potential_energy=elastic_potential_h, source=[0., 0., 0.],
                                                       plane_stress=True)
    nl_fragmented_pressure = BVP(domain=fvac, vform=f_heterogeneous_Hyperelastic_response, bc=frac_pressure)
    nl_whole = BVP(domain=hvac, vform=h_heterogeneous_Hyperelastic_response, bc=whole_pressure)
    
    file_name_p = f"../../data/out/fvac_Pressure_{TP}.xdmf"
    file_name_h = f"../../data/out/hvac_Pressure_{TP}.xdmf"
    
    nl_fragmented_pressure.solve(linear_solver='mumps', krylov_solver={'absolute_tolerance':1e-13}, absolute_tolerance=tolerance, relative_tolerance=tolerance)
    solu_frac = nl_fragmented_pressure.solution
    xdmf_save(path=file_name_p, solution=solu_frac, vform=f_heterogeneous_Hyperelastic_response)
    del nl_fragmented_pressure
    del solu_frac

    nl_whole.solve(linear_solver='mumps', krylov_solver={'absolute_tolerance':1e-13}, absolute_tolerance=tolerance, relative_tolerance=tolerance)
    solu_whole = nl_whole.solution
    xdmf_save(path=file_name_h, solution=solu_whole, vform=h_heterogeneous_Hyperelastic_response)
    del nl_whole
    del solu_whole